In [26]:
import os
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.tree import DecisionTreeRegressor
import xgboost
from gmdhpy import gmdh 
from sklearn.model_selection import RandomizedSearchCV
import pickle
import json
from tqdm import tqdm

In [2]:
def extract_time(time_str, pos):
    arr1 = time_str.split(' ')
    arr2 = arr1[0].split(':')
    prefix = arr1[1]
    hh = int(arr2[0])
    nn = int(arr2[1])
    ss = int(arr2[2])
    hhh = hh
    if prefix == 'PM':
        hhh += 12
        
    if pos == 1:
        return hhh
    elif pos == 2:
        return nn
    elif pos == 3:
        return ss
    else:
        res = ss + nn * 60 + hhh * 60 * 60
        return res

### Preparing the dataset

In [3]:
train = pd.read_csv('/home/osboxes/repos/sport/competitions_sendy/data/raw/Train.csv', sep=',') 
test = pd.read_csv('/home/osboxes/repos/sport/competitions_sendy/data/raw/Test.csv', sep=',') 
riders = pd.read_csv('/home/osboxes/repos/sport/competitions_sendy/data/raw/Riders.csv', sep=',')

In [4]:
train = pd.merge(train, riders, how='left', left_on='Rider Id', right_on='Rider Id', left_index=True)
test = pd.merge(test, riders, how='left', left_on='Rider Id', right_on='Rider Id', left_index=True)

In [5]:
x_cols = ['Order No', 'No_Of_Orders', 'Age', 'Average_Rating', 'No_of_Ratings', 'Platform Type', 'Pickup - Day of Month', 'Pickup - Weekday (Mo = 1)', 'Arrival at Pickup - Day of Month', 'Arrival at Pickup - Weekday (Mo = 1)', 'Distance (KM)', 'Temperature', 'Pickup Lat', 'Pickup Long', 'Destination Lat', 'Destination Long']

In [6]:
X_train = train[x_cols]
X_test = test[x_cols]
#Replace missing values with median values
#X_train = X_train.fillna(X_train.median())
#X_test = X_test.fillna(X_test.median())

In [7]:
%%time

# Calculated cols
X_train['Precipitation'] = train['Precipitation in millimeters'].fillna(0)
X_test['Precipitation'] = test['Precipitation in millimeters'].fillna(0)

CPU times: user 14.6 ms, sys: 1.11 ms, total: 15.7 ms
Wall time: 277 ms


/home/osboxes/envs/sport/lib64/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/osboxes/envs/sport/lib64/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [9]:
def process_time(col, new_col_prefix):
    X_train[new_col_prefix + '_hhh'] = train.apply(lambda x: extract_time(x[col],1), axis=1)
    X_train[new_col_prefix + '_nn'] = train.apply(lambda x: extract_time(x[col],2), axis=1)
    X_train[new_col_prefix + '_ss'] = train.apply(lambda x: extract_time(x[col],3), axis=1)
    X_train[new_col_prefix + '_timeall'] = train.apply(lambda x: extract_time(x[col],0), axis=1)

    X_test[new_col_prefix + '_hhh'] = test.apply(lambda x: extract_time(x[col],1), axis=1)
    X_test[new_col_prefix + '_nn'] = test.apply(lambda x: extract_time(x[col],2), axis=1)
    X_test[new_col_prefix + '_ss'] = test.apply(lambda x: extract_time(x[col],3), axis=1)
    X_test[new_col_prefix + '_timeall'] = test.apply(lambda x: extract_time(x[col],0), axis=1)

In [10]:
%%time
process_time('Pickup - Time', 'pickup_')

/home/osboxes/envs/sport/lib64/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/osboxes/envs/sport/lib64/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/osboxes/envs/sport/lib64/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using

CPU times: user 7.16 s, sys: 50.1 ms, total: 7.21 s
Wall time: 7.5 s


/home/osboxes/envs/sport/lib64/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [12]:
%%time
process_time('Arrival at Pickup - Time', 'arrival_pickup_')

/home/osboxes/envs/sport/lib64/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/osboxes/envs/sport/lib64/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/osboxes/envs/sport/lib64/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using

CPU times: user 7.13 s, sys: 70.1 ms, total: 7.2 s
Wall time: 7.3 s


/home/osboxes/envs/sport/lib64/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [15]:
#list(zip(X_train.columns, X_test.columns))

## Add speed

In [17]:
def make_speed(json_data):
    j1 = json_data['routes'][0]['summary']
    distance = j1['distance']
    duration = j1['duration']
    res = float(distance) / float(duration)
    return distance, duration, res * 1000

In [18]:
%%time
with open('/home/osboxes/repos/sport/competitions_sendy/data/external/coords_1.pkl', 'rb') as f:
    coords_1 = pickle.load(f)

CPU times: user 273 ms, sys: 152 ms, total: 425 ms
Wall time: 436 ms


In [19]:
assert len(coords_1) == train.shape[0] + test.shape[0]

In [20]:
len(coords_1)

28269

In [27]:
list_distance, list_duration, list_speed = [], [], []
for i in tqdm(range(len(coords_1))):
    j1 = json.loads(coords_1[i])
    distance, duration, speed = make_speed(j1)
    list_distance.append(distance)
    list_duration.append(duration)
    list_speed.append(speed)

100%|██████████| 28269/28269 [00:09<00:00, 2905.07it/s]


In [29]:
len(list_distance), len(list_duration), len(list_speed)

(28269, 28269, 28269)

In [30]:
train.shape[0], len(list_distance[:train.shape[0]])

(21201, 21201)

In [31]:
test.shape[0], len(list_distance[train.shape[0]:])

(7068, 7068)

In [32]:
%%time
X_train['dist_2'] = list_distance[:train.shape[0]]
X_test['dist_2'] = list_distance[train.shape[0]:]

CPU times: user 13.9 ms, sys: 0 ns, total: 13.9 ms
Wall time: 14.5 ms


/home/osboxes/envs/sport/lib64/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/osboxes/envs/sport/lib64/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [36]:
%%time
X_train['time_2'] = list_duration[:train.shape[0]]
X_test['time_2'] = list_duration[train.shape[0]:]

CPU times: user 14.2 ms, sys: 0 ns, total: 14.2 ms
Wall time: 14.8 ms


/home/osboxes/envs/sport/lib64/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/osboxes/envs/sport/lib64/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [38]:
%%time
X_train['speed_2'] = list_speed[:train.shape[0]]
X_test['speed_2'] = list_speed[train.shape[0]:]

CPU times: user 18 ms, sys: 0 ns, total: 18 ms
Wall time: 18 ms


/home/osboxes/envs/sport/lib64/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/osboxes/envs/sport/lib64/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [41]:
X_train = X_train.set_index('Order No')
X_test = X_test.set_index('Order No')

In [42]:
y_train = train[['Time from Pickup to Arrival']]

# Save datasets

In [44]:
X_train.to_csv('/home/osboxes/repos/sport/competitions_sendy/data/processed/train_2.csv', sep=',')

In [45]:
X_test.to_csv('/home/osboxes/repos/sport/competitions_sendy/data/processed/test_2.csv', sep=',')

In [46]:
y_train.to_csv('/home/osboxes/repos/sport/competitions_sendy/data/processed/target_2.csv', sep=',')